### Create Character Vector Database

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_chroma import Chroma
from langchain_cohere import ChatCohere

cohere_chat_model = ChatCohere(cohere_api_key="lMWcC83xVJlxEE5RahrziIdTiVqGetOp7Ba9YtD4")
cohere_embeddings = CohereEmbeddings(cohere_api_key="lMWcC83xVJlxEE5RahrziIdTiVqGetOp7Ba9YtD4",
                              user_agent='langchain',
                              model="embed-english-v2.0")

# Set the game name and character name
game_name = 'Honkai_Star_Rail'
character_name = 'Acheron'

# Load the file
with open(f'{game_name}/characters/{character_name}/character_knowledge.txt', errors='ignore') as f:
    public_info = f.read()

# Create Text Splitter, Chunk size is number of characters in each item of the list 
# While chunk overlap is how many characters from the previous item should be repeated in the next item 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap  = 0,
    length_function = len,
    separators = ['\n\n\n', '\n\n', '\n', '.', ',', ' ']
)

# Split the text 
public_info_list = text_splitter.split_text(public_info)


# Path must be inside the character folder in the game folder with the name vectordb
persist_directory = f'{game_name}/characters/{character_name}/vectordb'

# Create the Vector Database
vectordb = Chroma.from_texts(public_info_list, cohere_embeddings, persist_directory=persist_directory)

d:\University-Coding\Text-Gen-Proj\.venv\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


### Test the Vector Database

In [4]:
# search for a topic and see if you get relevant information, k is for number of similar items to return
docs = vectordb.similarity_search("Rogue", k = 1)
similar_story_string = "\n".join(doc.page_content for doc in docs)
print(similar_story_string)

...She had traversed numerous worlds before our paths crossed, and she possessed a wide range of life skills. Together, we cooked and established camps amidst the wilderness. On starless nights, she lent an ear to my stories and deliberated plans for the following day. Yet, more often than not, we journeyed in silence. The snow here bears a purplish-red hue, carrying the tangy flavor of raspberries. While she has almost lost her sense of taste, she appreciates the Orkron snow doughnuts I prepare...


### To add text to existing vector database 
#### (Optional, only if you want to add something that was not previously there in the above text file)

In [ ]:
# from langchain.embeddings import CohereEmbeddings
# import json
# import random
# from langchain.vectorstores import Chroma

#selected_key = json.load(open('apikeys.json', 'r'))['api_keys'][random.randint(
#    0, len(json.load(open('apikeys.json', 'r'))['api_keys'])-1)]
#embeddings = CohereEmbeddings(cohere_api_key=selected_key)
#persist_directory = 'Cyberpunk_2077/characters/Jackie_Welles/vectordb'
#vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
#vectordb.add_texts(["pineapples are yummy", "tomatoes are evil"])
#vectordb.persist()

#### To delete vectordb entirely, delete the public_vectordb folder inside the game folder

In [1]:
import shutil
# path to vectordb inside your character
folder_path = 'Honkai_Star_Rail/characters/Acheron/vectordb'
shutil.rmtree(folder_path)